In [8]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import Counter
import os
import dotenv

In [13]:
# Load the county growth data
growth_data = pd.read_csv('../Weather/growth_rates_with_environmental_risk.csv')

In [ ]:
def get_spotify_credentials():
    client_id = os.getenv('SPOTIPY_CLIENT_ID')
    client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
    
    if not client_id:
        client_id = input("Please enter your Spotify Client ID: ")
    if not client_secret:
        client_secret = input("Please enter your Spotify Client Secret: ")
    
    return client_id, client_secret

# Get Spotify API credentials
client_id, client_secret = get_spotify_credentials()

# Spotify API setup
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [15]:
def get_spotify_credentials():
    client_id = os.getenv('SPOTIPY_CLIENT_ID')
    client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
    
    if not client_id:
        client_id = input("Please enter your Spotify Client ID: ")
    if not client_secret:
        client_secret = input("Please enter your Spotify Client Secret: ")
    
    return client_id, client_secret

# Get Spotify API credentials
client_id, client_secret = get_spotify_credentials()

# Spotify API setup
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# Sort counties by a combined score (you can adjust this based on your priorities)
growth_data['CombinedScore'] = growth_data['Income Growth'] + growth_data['CAGR'] - growth_data['Normalized Risk Score']
top_counties = growth_data.sort_values('CombinedScore', ascending=False).head(10)

def get_top_genres_for_county(county_name, state):
    # Search for playlists related to the county
    query = f"{county_name} {state}"
    results = sp.search(q=query, type='playlist', limit=5)
    
    all_genres = []
    for playlist in results['playlists']['items']:
        tracks = sp.playlist_tracks(playlist['id'], fields='items.track.artists.id', limit=50)
        
        for item in tracks['items']:
            if item['track'] and item['track']['artists']:
                artist_id = item['track']['artists'][0]['id']
                artist_info = sp.artist(artist_id)
                all_genres.extend(artist_info['genres'])
    
    # Count genres and get top 5
    genre_counts = Counter(all_genres)
    return genre_counts.most_common(5)

# Get top genres for each top county
top_genres = {}
for _, county in top_counties.iterrows():
    county_name = county['GeoName'].split(',')[0].strip()
    state = county['GeoName'].split(',')[1].strip()
    print(f"Fetching genres for {county_name}, {state}...")
    top_genres[county['GeoName']] = get_top_genres_for_county(county_name, state)

# Print results
print("\nTop Genres for Top 10 Counties:")
for county, genres in top_genres.items():
    print(f"\n{county}:")
    for genre, count in genres:
        print(f"  {genre}: {count}")

# Add top genre to the dataframe
top_counties['TopGenre'] = top_counties['GeoName'].map(lambda x: top_genres[x][0][0] if top_genres[x] else 'N/A')

# Display and save results
print("\nTop 10 Counties with Their Top Genre:")
print(top_counties[['GeoName', 'Income Growth', 'CAGR', 'Normalized Risk Score', 'TopGenre']])

top_counties.to_csv('top_counties_with_genres.csv', index=False)
print("\nResults saved to 'top_counties_with_genres.csv'")

Fetching genres for Canyon, ID...
Fetching genres for Deschutes, OR...
Fetching genres for Pinal, AZ...
Fetching genres for Washington, UT...
Fetching genres for Kootenai, ID...
Fetching genres for Gallatin, MT...
Fetching genres for Palm Beach, FL...
Fetching genres for Sarasota, FL...
Fetching genres for Williamson, TN...
Fetching genres for Martin, FL...

Top Genres for Top 10 Counties:

Canyon, ID:
  classic rock: 72
  singer-songwriter: 60
  rock: 55
  soft rock: 55
  mellow gold: 54

Deschutes, OR:
  pop: 23
  glitch hop: 17
  rock: 11
  livetronica: 10
  manitoba indie: 10

Pinal, AZ:
  rap: 57
  filmi: 42
  modern bollywood: 37
  hip hop: 31
  atl hip hop: 22

Washington, UT:
  modern rock: 39
  stomp and holler: 37
  rock: 31
  folk-pop: 30
  rap: 24

Kootenai, ID:
  vocaloid: 47
  pop rap brasileiro: 37
  brazilian hip hop: 28
  anime rock: 25
  trap brasileiro: 13

Gallatin, MT:
  classic rock: 26
  mellow gold: 20
  soft rock: 18
  country rock: 17
  album rock: 17

Palm Be